In [15]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

#file0 = "/Users/apple/Workspace/intelliJ/jockey/regression_20190127.csv"

#dataset0 = pd.read_csv(file0)

#'horse_winCount','jockey_winCount',,'declaredHorseWeight','addedWeight'

features=['draw','horse_winPer',
                 'jockey_winPer','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRadio','weightRD']
target_fea='finishTime'
target=[target_fea]
sql = 'select raceDate,raceSeqOfDay,distance,raceClass, horseId, place,lbs,winOdds,'
for aFeature in features:
    sql = sql + aFeature +", "
sql = sql+ target_fea+ " from racecard"
#where racedate='20190213'"

columns=['raceDate','raceSeqOfDay','distance','raceClass',*features,*target]

dataset0 = pd.read_sql(sql,mydb)
print(dataset0)
allDates =['20190217']
#allDates= dataset0.raceDate.unique()
tableName='racecard'
#dataset0
updatedate='20190219'

       raceDate  raceSeqOfDay  distance  raceClass horseId  place    lbs  \
0      20181104             1      1200          4    B393      1   0.00   
1      20181104             1      1200          4    B250      2   0.05   
2      20181104             1      1200          4    B438      3   0.75   
3      20181104             1      1200          4    B178      4   1.75   
4      20181104             1      1200          4    B423      5   2.00   
5      20181104             1      1200          4    B242      6   2.75   
6      20181104             1      1200          4    B367      7   3.75   
7      20181104             1      1200          4    B365      8   3.75   
8      20181104             1      1200          4    B403      9   5.25   
9      20181104             1      1200          4    B448     10   5.25   
10     20181104             1      1200          4    B275     11   6.50   
11     20181104             2      1000          4    S397      1   0.00   
12     20181

In [16]:

for raceDate in allDates:
    #if(raceDate <20180218):
    #     continue
    print(raceDate)
    dataset_ofADay = dataset0[(dataset0['raceDate']==raceDate)]
    
    allSeqs = dataset_ofADay.raceSeqOfDay.unique()

    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset_ofADay['raceSeqOfDay']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(raceDate,seq,distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['raceDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        y_test=dataset_test[target] #lbw


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #        % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))
        #print('explained_variance_score score: %.2f' % explained_variance_score(y_test, y_test_predicted))
         #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId','place','lbs','finishTime']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #print(dataset_test_withPredicted)
       
        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['raceDate'])
            raceSeqOfDate =str(row['raceSeqOfDay'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_date=" + updatedate +", predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )


20190217
20190217 1 2000 4
1
1  record(s) updated 1 , V407
2
1  record(s) updated 1 , V192
3
1  record(s) updated 1 , B137
4
1  record(s) updated 1 , S015
5
1  record(s) updated 1 , S147
6
1  record(s) updated 1 , V039
7
1  record(s) updated 1 , A325
8
1  record(s) updated 1 , A358
9
1  record(s) updated 1 , A284
10
1  record(s) updated 1 , V369
20190217 2 1000 4
1
1  record(s) updated 2 , C229
2
1  record(s) updated 2 , A130
3
1  record(s) updated 2 , C224
4
1  record(s) updated 2 , C226
5
1  record(s) updated 2 , C034
6
1  record(s) updated 2 , B367
7
1  record(s) updated 2 , B177
8
1  record(s) updated 2 , C105
9
1  record(s) updated 2 , C130
10
1  record(s) updated 2 , C086
11
1  record(s) updated 2 , T392
12
1  record(s) updated 2 , B214
13
1  record(s) updated 2 , C131
14
1  record(s) updated 2 , C134
20190217 3 1400 4
1
1  record(s) updated 3 , A088
2
1  record(s) updated 3 , B225
3
1  record(s) updated 3 , B460
4
1  record(s) updated 3 , C007
5
1  record(s) updated 3 , A057
6
1

In [47]:
 #explained_variance_score(y_test, y_test_predicted)
df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
dataset_test_toshow=dataset_test[['rateDate','raceSeqOfDate','distance','raceClass','horseId','place','winOdds','lbw','finishTime']]
dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
dataset_test_withPredicted

,rateDate,raceSeqOfDate,distance,raceClass,horseId,place,winOdds,lbw,finishTime,finishTime_predicted
13560,20190123,8,1650,3,B085,3,4.1,0.50,100.09,100.021965
13558,20190123,8,1650,3,A228,1,3.2,0.00,100.02,100.046780
13559,20190123,8,1650,3,A240,2,4.3,0.15,100.05,100.390609
13562,20190123,8,1650,3,B223,5,5.6,2.00,100.33,100.393883
13564,20190123,8,1650,3,A189,7,37.0,3.00,100.49,100.498137
13565,20190123,8,1650,3,A362,8,21.0,5.25,100.85,100.502601
13566,20190123,8,1650,3,V279,9,20.0,5.75,100.95,100.622227
13563,20190123,8,1650,3,A247,6,15.0,2.75,100.48,100.655952
13561,20190123,8,1650,3,V379,4,42.0,1.50,100.26,100.683624
13568,20190123,8,1650,3,T098,11,86.0,9.50,101.56,100.695285
